### Add text feature

1. number of words
2. avg word length
3. number of sentence
4. avg number of words in a sentence

### Use top 1000 unigram and Tf Idf

In [1]:
import string
sent_list = []
labels = []
with open('balance_train.tsv', 'r') as f:
    for line in f:
        sent, label = line.strip('\n').split('\t')
        labels.append(int(label))
        words = sent.split(' ')
        pure_words = []
        for w in words:
            if w not in string.punctuation:
                pure_words.append(w)
        sent_list.append(pure_words)

In [2]:
from collections import defaultdict
unigram = defaultdict(int)
for sent in sent_list:
    for w in sent:
        unigram[w] += 1
unigram = sorted(unigram.items(), key=lambda x:x[1], reverse=True)

In [3]:
top1000_unigram = unigram[:1000]
uniwords = [i[0] for i in top1000_unigram]
uniwordId = dict(zip(uniwords, range(len(uniwords))))

In [4]:
def calc_tf(word, sent):
    sent_len = len(sent)
    word_count = sent.count(word)
    tf = word_count/sent_len
    return tf

In [5]:
import math
def calc_idf(word, sents):
    doc_num = len(sents)
    word_count = 0
    for s in sents:
        if word in s:
            word_count += 1
    return math.log(doc_num/word_count, 10)

In [6]:
uniword_tfidf = defaultdict(dict)
for word in uniwords:
    uniword_tfidf[word]['idf'] = calc_idf(word, sent_list)
sentId = 0
for sent in sent_list:
    for word in sent:
        if word in uniwords:
            uniword_tfidf[word][sentId] = calc_tf(word, sent)
    sentId += 1

In [7]:
train_X = []
sentId = 0
for sent in sent_list:
    feat = [0]*len(uniwords)
    for word in sent:
        if word in uniwords:
            idf = uniword_tfidf[word]['idf']
            tf = uniword_tfidf[word][sentId]
            feat[uniwordId[word]] = tf*idf
    train_X.append(feat)
    sentId += 1

In [8]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(train_X, labels)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
theta = clf.coef_[0]

In [10]:
weights = list(zip(theta, uniwords + ['constant_feat']))

In [11]:
weights.sort()

In [12]:
print(weights[:10])

[(-13.284070698683689, 'return'), (-12.660907179679352, 'disappoint'), (-12.57235126798869, "n't"), (-11.760267543605465, 'not'), (-9.282087864652958, 'wast'), (-9.155801581836393, 'poor'), (-8.965083058001548, 'howev'), (-8.067596958535928, 'unfortun'), (-7.512432452440564, 'useless'), (-6.836375038275021, 'worst')]


In [13]:
print(weights[-10:])

[(7.078804798393313, 'favorit'), (7.524372259001453, 'best'), (8.034549606799494, 'awesom'), (8.268230749297278, 'excel'), (9.160219776882153, 'easi'), (9.376639017778011, 'perfectli'), (9.402134420121362, 'highli'), (10.775171638638696, 'perfect'), (13.880447133070883, 'great'), (15.561025823615415, 'love')]


In [14]:
positiveWords = []
negativeWords = []
for item in weights[:10]:
    w = item[1]
    negativeWords.append(w)
for item in weights[-10:]:
    w = item[1]
    positiveWords.append(w)

In [15]:
test_sent_list = []
test_labels = []
with open('balance_test.tsv', 'r') as f:
    for line in f:
        sent, label = line.strip('\n').split('\t')
        test_labels.append(int(label))
        words = sent.split(' ')
        pure_words = []
        for w in words:
            if w not in string.punctuation:
                pure_words.append(w)
        test_sent_list.append(pure_words)

In [17]:
test_uniword_tfidf = defaultdict(dict)
for word in uniwords:
    test_uniword_tfidf[word]['idf'] = calc_idf(word, test_sent_list)
sentId = 0
for sent in test_sent_list:
    for word in sent:
        if word in uniwords:
            test_uniword_tfidf[word][sentId] = calc_tf(word, sent)
    sentId += 1

In [18]:
test_X = []
sentId = 0
for sent in test_sent_list:
    feat = [0]*len(uniwords)
    for word in sent:
        if word in uniwords:
            idf = test_uniword_tfidf[word]['idf']
            tf = test_uniword_tfidf[word][sentId]
            feat[uniwordId[word]] = tf*idf
    test_X.append(feat)
    sentId += 1

In [19]:
pred = clf.predict(test_X)

In [20]:
correct = 0
for y, p in zip(test_labels, list(pred)):
    if y == p:
        correct += 1
acc = correct/len(test_labels)
print(acc)

0.8218440954043048


### Add positive and negative words

In [21]:
posId = dict(zip(positiveWords, range(len(positiveWords))))
negId = dict(zip(negativeWords, range(len(negativeWords))))

In [22]:
new_train_X = []
for sent, vector in zip(sent_list, train_X):
    posVec = [0]*len(positiveWords)
    negVec = [0]*len(negativeWords)
    posAppear = set()
    negAppear = set()
    for w in sent:
        if w in positiveWords:
            posVec[posId[w]] += 1
            posAppear.add(w)
        if w in negativeWords:
            negVec[negId[w]] += 1
            negAppear.add(w)
    posFreq = len(posAppear)/len(positiveWords)
    negFreq = len(negAppear)/len(negativeWords)
    new_v = vector + posVec + [posFreq] + negVec + [negFreq]
    new_train_X.append(new_v)

In [23]:
print(len(new_train_X[0]))

1022


In [24]:
new_test_X = []
for sent, vector in zip(test_sent_list, test_X):
    posVec = [0]*len(positiveWords)
    negVec = [0]*len(negativeWords)
    posAppear = set()
    negAppear = set()
    for w in sent:
        if w in positiveWords:
            posVec[posId[w]] += 1
            posAppear.add(w)
        if w in negativeWords:
            negVec[negId[w]] += 1
            negAppear.add(w)
    posFreq = len(posAppear)/len(positiveWords)
    negFreq = len(negAppear)/len(negativeWords)
    new_v = vector + posVec + [posFreq] + negVec + [negFreq]
    new_test_X.append(new_v)

In [26]:
print(len(new_train_X))
print(len(new_test_X))

129046
27504


In [25]:
clf = LogisticRegression().fit(new_train_X, labels)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [27]:
pred = clf.predict(new_test_X)

In [28]:
correct = 0
for y, p in zip(test_labels, list(pred)):
    if y == p:
        correct += 1
acc = correct/len(test_labels)
print(acc)

0.8232984293193717


### Add text features

number of words in one sentence

In [50]:
new_train_X2 = []
for sent, vector in zip(sent_list, new_train_X):
    word_num = len(sent)
    new_v = vector + [word_num]
    new_train_X2.append(new_v)

In [51]:
new_test_X2 = []
for sent, vector in zip(test_sent_list, new_test_X):
    word_num = len(sent)
    new_v = vector + [word_num]
    new_test_X2.append(new_v)

In [52]:
clf = LogisticRegression().fit(new_train_X2, labels)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [53]:
pred = clf.predict(new_test_X2)

In [54]:
correct = 0
for y, p in zip(test_labels, list(pred)):
    if y == p:
        correct += 1
acc = correct/len(test_labels)
print(acc)

0.8223531122745782


avg word length

In [55]:
new_train_X2 = []
for sent, vector in zip(sent_list, new_train_X):
    word_len_sum = 0
    for w in sent:
        word_len_sum += len(w)
    if len(sent) != 0:
        avg_word_len = word_len_sum/len(sent)
    else:
        avg_word_len = 0
    new_v = vector + [avg_word_len]
    new_train_X2.append(new_v)

In [56]:
new_test_X2 = []
for sent, vector in zip(test_sent_list, new_test_X):
    word_len_sum = 0
    for w in sent:
        word_len_sum += len(w)
    if len(sent) != 0:
        avg_word_len = word_len_sum/len(sent)
    else:
        avg_word_len = 0
    new_v = vector + [avg_word_len]
    new_test_X2.append(new_v)

In [57]:
clf = LogisticRegression().fit(new_train_X2, labels)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [58]:
pred = clf.predict(new_test_X2)

In [59]:
correct = 0
for y, p in zip(test_labels, list(pred)):
    if y == p:
        correct += 1
acc = correct/len(test_labels)
print(acc)

0.8229348458406051


number of sentence

In [60]:
train_sent_count = []
with open('balance_train.tsv', 'r') as f:
    for line in f:
        sent, _ = line.strip('\n').split('\t')
        words = sent.split(' ')
        cur_s = 0
        for w in words:
            if w == '.' or w == '?' or w == '!' or w == ',':
                cur_s += 1
        if sent != "" and cur_s == 0:
            cur_s = 1
        train_sent_count.append(cur_s)

In [61]:
new_train_X2 = []
for sent_c, vector in zip(train_sent_count, new_train_X):
    new_v = vector + [sent_c]
    new_train_X2.append(new_v)

In [62]:
clf = LogisticRegression().fit(new_train_X2, labels)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [63]:
test_sent_count = []
with open('balance_test.tsv', 'r') as f:
    for line in f:
        sent, _ = line.strip('\n').split('\t')
        words = sent.split(' ')
        cur_s = 0
        for w in words:
            if w == '.' or w == '?' or w == '!' or w == ',':
                cur_s += 1
        if sent != "" and cur_s == 0:
            cur_s = 1
        test_sent_count.append(cur_s)

In [64]:
new_test_X2 = []
for sent_c, vector in zip(test_sent_count, new_test_X):
    new_v = vector + [sent_c]
    new_test_X2.append(new_v)

In [65]:
pred = clf.predict(new_test_X2)

In [66]:
correct = 0
for y, p in zip(test_labels, list(pred)):
    if y == p:
        correct += 1
acc = correct/len(test_labels)
print(acc)

0.8229348458406051


avg number of words in a sentence

In [67]:
train_avgword_count = []
with open('balance_train.tsv', 'r') as f:
    for line in f:
        sent, _ = line.strip('\n').split('\t')
        words = sent.split(' ')
        cur_word_c = 0
        cur_s_c = 0
        for w in words:
            if w == '.' or w == '?' or w == '!' or w == ',':
                cur_s_c += 1
            else:
                cur_word_c += 1
        if sent != "" and cur_s_c == 0:
            cur_s_c = 1
        if sent == "":
            train_avgword_count.append(0)
        else:
            train_avgword_count.append(cur_word_c/cur_s_c)

In [68]:
new_train_X2 = []
for sent_c, vector in zip(train_avgword_count, new_train_X):
    new_v = vector + [sent_c]
    new_train_X2.append(new_v)

In [69]:
test_avgword_count = []
with open('balance_test.tsv', 'r') as f:
    for line in f:
        sent, _ = line.strip('\n').split('\t')
        words = sent.split(' ')
        cur_word_c = 0
        cur_s_c = 0
        for w in words:
            if w == '.' or w == '?' or w == '!' or w == ',':
                cur_s_c += 1
            else:
                cur_word_c += 1
        if sent != "" and cur_s_c == 0:
            cur_s_c = 1
        if sent == "":
            test_avgword_count.append(0)
        else:
            test_avgword_count.append(cur_word_c/cur_s_c)

In [70]:
new_test_X2 = []
for sent_c, vector in zip(test_avgword_count, new_test_X):
    new_v = vector + [sent_c]
    new_test_X2.append(new_v)

In [71]:
clf = LogisticRegression().fit(new_train_X2, labels)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [72]:
pred = clf.predict(new_test_X2)

In [73]:
correct = 0
for y, p in zip(test_labels, list(pred)):
    if y == p:
        correct += 1
acc = correct/len(test_labels)
print(acc)

0.8239528795811518
